#**Despliegue del Modelo de Regresión Logística con MLflow**

En este notebook se despliega un modelo de Regresión Logística para la predicción de enfermedad cardíaca utilizando MLflow.

Durante la fase anterior de modelamiento se evaluaron distintos algoritmos de clasificación y la Regresión Logística fue el modelo que
obtuvo el mejor desempeño global, por lo que se seleccionó como el modelo apropiado para desplegar y poner en producción.

## 1. Configuración del entorno y MLFlow

En principio, instalan las dependencias necesarias para habilitar el entorno de trabajo, incluyendo MLflow, scikit-learn, pandas y numpy. Adicionalmente, se inicia el servidor de MLflow directamente desde la sesión de Colab, configurándolo como un servicio local que actúa simultáneamente como servidor de seguimiento (tracking server) y repositorio de artefactos. Esta instancia de MLflow servirá para almacenar los registros experimentales y gestionar el ciclo de vida del modelo en producción.

In [1]:
!pip install -q mlflow requests pyngrok scikit-learn pandas numpy


In [2]:
import os
import mlflow
import pandas as pd
import numpy as np
from IPython.display import display
from pyngrok import ngrok


In [3]:
# lanzar servidor de MLflow (tracking + artifacts) en el puerto 5000
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)
print("Servidor de MLflow levantado en http://localhost:5000")


Servidor de MLflow levantado en http://localhost:5000


Es importante poder exponer el servidor de MLflow de forma pública, para ello se utiliza Ngrok. Esto es especialmente útil en entornos donde el servidor no puede ser accedido directamente, como en el presente caso dado que se está trabajando en Google Colab. En ese sentido, el túnel generado por Ngrok facilita la visualización del panel web de MLflow sin modificar la configuración del servidor subyacente.

In [4]:
!pip install -q pyngrok
from pyngrok import ngrok


In [5]:
token = "2zCaSMWeM6ryW9JUelXTzNR9plg_vivFsFet3GjNkXDvis3v"
os.environ["NGROK_TOKEN"] = token
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
mlflow_tunnel = ngrok.connect(5000, "http", host_header="localhost")
mlflow_tunnel

<NgrokTunnel: "https://43c7ffb3abad.ngrok-free.app" -> "http://localhost:5000">

## Configuración de MLflow y creación del experimento

En esta sección se establece la conexión explícita con el servidor MLflow levantado previamente. Posteriormente, se crea un experimento donde serán almacenados todos los runs asociados al modelo de Regresión Logística. Esto permite mantener una trazabilidad estructurada de los parámetros, métricas y artefactos generados durante el entrenamiento.

In [7]:
mlflow.set_tracking_uri("http://localhost:5000")

experiment_name = "heart_failure_logistic"
mlflow.set_experiment(experiment_name)

exp = mlflow.get_experiment_by_name(experiment_name)
exp_id = exp.experiment_id
exp_id


2025/12/11 00:02:30 INFO mlflow.tracking.fluent: Experiment with name 'heart_failure_logistic' does not exist. Creating a new experiment.


'1'

## Carga y preparación del conjunto de datos

Posteriormente, se incorpora el conjunto de datos desde la fuente original, que corresponde al repositorio de nuestro proyecto en Github, y se realizan las transformaciones necesarias para el entrenamiento del modelo. Se selecciona la variable objetivo, se aplican técnicas de codificación a variables categóricas mediante one-hot encoding, y finalmente se dividen los datos en conjuntos de entrenamiento y prueba, preservando la distribución de clases.

In [8]:
from sklearn.model_selection import train_test_split

# Carga de datos
url = "https://raw.githubusercontent.com/Dovallec/Heart_Failure_Analysis/master/data/heart.csv"
df = pd.read_csv(url)
display(df.head())

target_col = "HeartDisease"
X = df.drop(columns=[target_col])
y = df[target_col]

# Codificación de variables categóricas
cat_cols = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True)

# Para seguir el esquema del notebook de ejemplo, trabajamos con numpy arrays
features = X_encoded.to_numpy().astype(float)
labels = y.to_numpy()

features_train, features_test, labels_train, labels_test = train_test_split(
    features,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

features_train.shape, features_test.shape


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


((734, 15), (184, 15))

## Entrenamiento y registro del modelo de Regresión Logística

En esta fase se entrena el modelo de Regresión Logística utilizando los datos preprocesados y los hiperparámetros seleccionados durante la experimentación preliminar. El entrenamiento se ejecuta dentro de un contexto mlflow.start_run() para garantizar su registro automático en MLflow. Posteriormente, se almacenan métricas de desempeño, parámetros utilizados y el modelo entrenado. Finalmente, el modelo es registrado formalmente en el Model Registry, lo cual habilita su uso en despliegues futuros.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

with mlflow.start_run(
    run_name="logistic_regression_best_model",
    experiment_id=exp_id
):
    # Hiperparámetros seleccionados en la fase anterior de modelamiento
    model = LogisticRegression(
        penalty="l2",
        C=1.0,
        solver="lbfgs",
        max_iter=100000,
        class_weight="balanced",
        random_state=42
    )

    model.fit(features_train, labels_train)
    y_pred = model.predict(features_test)

    acc = accuracy_score(labels_test, y_pred)
    report = classification_report(labels_test, y_pred, output_dict=True)

    # Métricas principales
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision_macro", report["macro avg"]["precision"])
    mlflow.log_metric("recall_macro", report["macro avg"]["recall"])
    mlflow.log_metric("f1_macro", report["macro avg"]["f1-score"])

    # Parámetros del modelo
    mlflow.log_params({
        "penalty": "l2",
        "C": 1.0,
        "solver": "lbfgs",
        "max_iter": 100000,
        "class_weight": "balanced",
        "random_state": 42
    })

    # Registro del modelo + creación/actualización del Registered Model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="heart_failure_logistic"
    )

    print("Accuracy en test:", acc)


2025/12/11 00:03:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'heart_failure_logistic'.
2025/12/11 00:03:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: heart_failure_logistic, version 1
Created version '1' of model 'heart_failure_logistic'.
2025/12/11 00:03:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run logistic_regression_best_model at: http://localhost:5000/#/experiments/1/runs/1ea40fe740554120b2631741eeeaa3fa.
2025/12/11 00:03:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


Accuracy en test: 0.8913043478260869


## Configuración del tracking URI para el servidor de modelos y despliegue

En esta sección se establece la URI del servidor de MLflow que actuará como fuente para la recuperación del modelo registrado. De esta forma, el servidor de inferencia podrá acceder al modelo almacenado en el Model Registry. Posteriormente, se levanta el servidor de modelos utilizando mlflow models serve, exponiendo el modelo de Regresión Logística en un endpoint HTTP accesible desde cualquier cliente.

In [10]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
print("MLFLOW_TRACKING_URI:", os.environ["MLFLOW_TRACKING_URI"])


MLFLOW_TRACKING_URI: http://localhost:5000


In [11]:
command = """
mlflow models serve -m 'models:/heart_failure_logistic/1' -p 8001 --env-manager 'local' &
"""
get_ipython().system_raw(command)
print("API de MLflow levantada en http://localhost:8001/invocations")


API de MLflow levantada en http://localhost:8001/invocations


## Validación 1 -> Consumo del API con requests

En esta fase finalmente se valida el funcionamiento del servidor de inferencia enviando solicitudes HTTP al endpoint /invocations. Para ello, se construye una carga útil en formato JSON utilizando registros del conjunto de prueba. Las características se convierten a tipos nativos de Python para garantizar compatibilidad con el formato JSON.

In [12]:
import requests

data_request = features_test[:2].tolist()
data_request


[[46.0,
  115.0,
  0.0,
  0.0,
  113.0,
  1.5,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0],
 [58.0,
  132.0,
  224.0,
  0.0,
  173.0,
  3.2,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0]]

In [13]:
r = requests.post(
    "http://localhost:8001/invocations",
    json={"inputs": data_request}
)

print("Código de estado:", r.status_code)
print("Respuesta del modelo:", r.text)


Código de estado: 200
Respuesta del modelo: {"predictions": [1, 0]}


El resultado de esta validación del servicio mostró un código de estado HTTP 200, lo cual indica que el servidor de MLflow recibió, procesó y respondió correctamente a la solicitud JSON enviada. La respuesta { "predictions": [1, 0] } evidencia que el modelo de Regresión Logística almacenado en el Model Registry fue cargado exitosamente y ejecutó inferencia sobre las dos instancias proporcionadas. Este resultado confirma que el despliegue es funcional y estable.

## Validación 2 -> Uso del modelo con datos nuevos para simular entrada a producción

Con el fin de realizar una validación adicional y completamente independiente del conjunto de prueba, se construyeron manualmente nuevos datos que simulan potenciales entradas reales en un entorno de producción. Esta verificación es fundamental para evaluar no solo la capacidad predictiva del modelo, sino también la correcta operación del servicio desplegado. Al enviar estos datos inéditos al endpoint de inferencia y obtener una respuesta válida, se confirma que el modelo se encuentra efectivamente operativo, accesible y capaz de generar predicciones coherentes en un escenario que replica condiciones reales de uso.

### Caso 1: Persona que tiene indicadores de salud deficientes

In [14]:
sample_new = [
    58,      # Age
    138,     # RestingBP
    237,     # Cholesterol
    1,       # FastingBS
    155,     # MaxHR
    1.2,     # Oldpeak
    1,   # Sex_M
    0,   # ChestPainType_ATA
    1,   # ChestPainType_NAP
    0,   # ChestPainType_TA
    0,   # RestingECG_LVH
    1,   # RestingECG_Normal
    1,   # ExerciseAngina_Y
    1,   # ST_Slope_Flat
    0    # ST_Slope_Up
]


In [15]:
import requests

payload = {
    "inputs": [sample_new]
}

r = requests.post(
    "http://localhost:8001/invocations",
    json=payload
)

print("Código de estado:", r.status_code)
print("Respuesta del modelo:", r.text)


Código de estado: 200
Respuesta del modelo: {"predictions": [1]}


### Caso 2: Persona que tiene mejores condiciones de salud que el caso 1

In [16]:
sample_low_risk = [
    35,     # Age
    118,    # RestingBP (normal)
    180,    # Cholesterol (moderado/bajo)
    0,      # FastingBS (normal)
    165,    # MaxHR (alto, típico en personas jóvenes y sanas)
    0.1,    # Oldpeak (bajo riesgo)
    1,  # Sex_M
    1,  # ChestPainType_ATA (bajo riesgo)
    0,  # ChestPainType_NAP
    0,  # ChestPainType_TA
    0,  # RestingECG_LVH
    1,  # RestingECG_Normal
    0,  # ExerciseAngina_Y (no angina)
    0,  # ST_Slope_Flat
    1   # ST_Slope_Up (favorable)
]


In [17]:
import requests

payload = {
    "inputs": [sample_low_risk]
}

r = requests.post(
    "http://localhost:8001/invocations",
    json=payload
)

print("Código de estado:", r.status_code)
print("Respuesta del modelo:", r.text)


Código de estado: 200
Respuesta del modelo: {"predictions": [0]}


Los resultados obtenidos evidencian que el despliegue fue exitoso y que el sistema responde adecuadamente ante entradas nuevas, lo cual demuestra un funcionamiento estable y confiable del modelo en su entorno de servicio.

Adicionalmente, el modelo de clasificación evidencia un comportamiento congruente con las expectativas clínicas y operativas. En un escenario de producción (ejemplos de la Validación 2), al suministrar manualmente datos que representan a una persona con indicadores desfavorables de salud, el modelo predice correctamente la presencia de enfermedad cardíaca (clase 1). De manera consistente, cuando se introducen datos que corresponden a un perfil saludable, el modelo predice la ausencia de la enfermedad (clase 0). Estos resultados confirman que el modelo no solo está correctamente desplegado, sino que también responde de forma coherente ante nuevos casos, reproduciendo de manera adecuada el patrón subyacente aprendido durante el entrenamiento.